In [1]:
import pandas as pd

# Validation of TRI Modeling

- only three sensors in reasonable proximity for 90-99 and all lead
- want to check the 2000-2018 for potential matches

In [6]:
TRI_base_process_90_18_nopubchem_df = pd.read_csv('/home/boogie2/Hanson_Lab/TRI_STILT/data/processed/TRI_base_process_90_18_nopubchem.csv')
TRI_base_process_90_18_nopubchem_df = TRI_base_process_90_18_nopubchem_df.drop(columns = ['Unnamed: 0'])

In [7]:
TRI_base_process_90_18_nopubchem_df.shape

(2244, 31)